In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

WEAVIATE_CLOUD_URL = os.getenv("WEAVIATE_CLOUD_URL")
WEAVIATE_CLOUD_KEY = os.getenv("WEAVIATE_CLOUD_KEY_READ")

## Connect

In [ ]:
import weaviate, json

# Connect to a Weaviate Cloud instance
client = weaviate.connect_to_wcs(
    cluster_url=WEAVIATE_CLOUD_URL,
    auth_credentials=weaviate.auth.AuthApiKey(WEAVIATE_CLOUD_KEY),
)

# Connect to the local instance deployed with Docker Compose
# client = weaviate.connect_to_local()

client.is_ready()

## Create Tenant-ready collection

In [ ]:
from weaviate.classes.config import Configure

if (client.collections.exists("PapersTen")):
    client.collections.delete("PapersTen")

client.collections.create(
    "PapersTen",

    vectorizer_config=Configure.Vectorizer.text2vec_aws(
        model="amazon.titan-embed-text-v1",
        # region="eu-central-1",
        region="us-east-1",
    ),
    
    generative_config=Configure.Generative.aws(
        region="us-east-1",
        model="cohere.command-r-plus-v1:0"
    ),

    multi_tenancy_config=Configure.multi_tenancy(True)
)

## List Tenants

In [ ]:
papers = client.collections.get("PapersTen")
papers.tenants.get()

## Close the client when done

In [ ]:
client.close()